In [1]:
import pandas as pd
import uuid
#from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

# Used in FL
import nest_asyncio
nest_asyncio.apply()
import collections
import tensorflow_federated as tff

np.random.seed(0)

2023-04-12 18:21:52.423733: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 18:21:52.627863: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-12 18:21:52.629271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 18:21:58.642584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Testing TFF
# tff.backends.native.set_local_python_execution_context()
print(tff.federated_computation(lambda: 'Hello, World!')())

2023-04-12 18:22:15.627943: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-12 18:22:15.629078: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
E0412 18:22:15.880977900     207 socket_utils_common_posix.cc:221] check for SO_REUSEPORT: {"created":"@1681338135.880942100","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":199,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
E0412 18:22:15.881804100     207 socket_utils_common_posix.cc:327] setsockopt(TCP_USER_TIMEOUT) Protocol not available


b'Hello, World!'


In [3]:
# Read the CSV files into separate dataframes
df_ey = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_younger.csv')
df_eo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_older.csv')
df_ly = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_younger.csv')
df_lo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_older.csv')

In [4]:
# Concatenate the dataframes into one dataframe with matching headers
df = pd.concat([df_ey, df_eo, df_ly, df_lo], ignore_index=True, sort=False)

In [5]:
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
0,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
1,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
3,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
870,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
871,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0
872,1,1,3,2,1,1,48.0,0.565451,13.000000,0.310525,191.393162,3.060358,1.0,NoDx,55.0


In [6]:
# Generate a unique client ID for each row
df['client_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Set the index of the dataframe to the client_id column
df.set_index('client_id', inplace=True)

In [7]:
# Print the dataframe
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
client_id,,,,,,,,,,,,,,,
23efe735-c134-4ffd-841b-e67e3f618d28,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
405d15dc-f781-4aed-abcb-288b2674dc38,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2fd0705a-a8fe-49db-9426-e6fc7e9378f6,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
a46ea600-2cda-428e-aa8c-8f35ca84abb0,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
697524fa-f10a-42ed-9ed3-52eaee0f1b32,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8fb6410d-989d-4321-8cb3-d952001a2216,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
f1c9a021-7770-4834-b7ce-49da4a0151d7,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
542bc78d-d5a2-44b2-ad27-44b879f852b1,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0


In [8]:
# Get the columns with NaN values
nan_cols = df.columns[df.isnull().any()]

# Print the columns with NaN values as a list
print(nan_cols.tolist())

['GramComplex']


In [9]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Fill missing values in the numeric columns with the mean value
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [10]:
# Replace the 'NoDx' value with 0 and the 'Dx' value with 1
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].replace("NoDx", 0).replace("Dx", 1)
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].astype(int)

In [11]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
print(len(train_data), len(test_data), type(train_data))

699 175 <class 'pandas.core.frame.DataFrame'>


In [13]:
print(test_data.dtypes)

Level.gender             int64
Level.ethnic             int64
Level.med                int64
Level.fed                int64
Level.income             int64
FamHist                  int64
Percentile             float64
GCC                    float64
Degree                 float64
HC                     float64
Between                float64
PathLength             float64
CombWords              float64
AnyLangorReadDxOnly      int64
GramComplex            float64
dtype: object


In [22]:
# Convert the training data to a TensorFlow Dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="AnyLangorReadDxOnly")
print(train_ds)#.element_spec

<_PrefetchDataset element_spec=({'Level.gender': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.ethnic': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.med': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.fed': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Level.income': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'FamHist': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Percentile': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GCC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Degree': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'HC': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Between': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'PathLength': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'CombWords': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'GramComplex': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(N

In [ ]:
# ------------------------------------------------------Federated Learning Model------------------------------------------------

In [17]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    
    return collections.OrderedDict(
        # Reshape pixels for this Digit from a 28x28 2D array, into a 1D array with 784 pixels
        # and return the (Label, [Pixels]) as an `OrderedDict`.
        x=tf.reshape(element['Level.gender'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))
  # Return reformated, batched, shuffled OrderedDicts of each Digit in the given client dataset
  # prefetch is used to fetch a set of batches (10 in this case) in order to speed up the ML processing.
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [25]:
preprocessed_example_dataset = train_ds.shuffle(SHUFFLE_BUFFER, seed=1)

In [ ]:
#--------------------------------------------Rest of Centralized Model----------------------------------------------------------

In [17]:
model = tfdf.keras.RandomForestModel(num_trees=50, max_depth=10)

Use /tmp/tmpusxazvrv as temporary training directory


In [18]:
# Fit the model to the training data
model.fit(train_ds)

Reading training dataset...


2023-04-08 19:05:17.204310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int64 and shape [699]
	 [[{{node Placeholder/_14}}]]


Training dataset read in 0:00:02.906207. Found 699 examples.
Training model...
Model trained in 0:00:00.070871
Compiling model...


[INFO 23-04-08 19:05:20.1411 EDT kernel.cc:1242] Loading model from path /tmp/tmpusxazvrv/model/ with prefix a4f768fc2d534889
[INFO 23-04-08 19:05:20.1710 EDT decision_forest.cc:660] Model loaded with 50 root(s), 2702 node(s), and 14 input feature(s).
[INFO 23-04-08 19:05:20.1711 EDT abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-04-08 19:05:20.1711 EDT kernel.cc:1074] Use fast generic engine
2023-04-08 19:05:20.189196: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype double and shape [699]
	 [[{{node Placeholder/_6}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [19]:
# Convert the testing data to a TensorFlow Dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)

# Make predictions on the testing data
y_pred = model.predict(test_ds)

2023-04-08 19:05:21.215846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [175]
	 [[{{node Placeholder/_8}}]]


1/1 [==============================] - 0s 82ms/step


In [20]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=2)

In [21]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (14):
	Between
	CombWords
	Degree
	FamHist
	GCC
	GramComplex
	HC
	Level.ethnic
	Level.fed
	Level.gender
	Level.income
	Level.med
	PathLength
	Percentile

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.   "Percentile"  0.280091 ################
    2.    "CombWords"  0.236790 ##########
    3.  "GramComplex"  0.218871 ########
    4.          "GCC"  0.192062 #####
    5.           "HC"  0.189847 #####
    6.       "Degree"  0.188856 #####
    7.      "Between"  0.185558 ####
    8.   "PathLength"  0.183286 ####
    9.      "FamHist"  0.157640 #
   10.    "Level.fed"  0.155451 #
   11. "Level.income"  0.150453 
   12.  

In [22]:
model.compile(metrics=["accuracy"])
evaluation = model.evaluate(test_data, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

ValueError: in user code:

    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 518, in test_function  *
        return step_function_trained(self, iterator)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 497, in run_step  *
        outputs = model.test_step(data)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1788, in test_step  **
        y_pred = self(x, training=False)
    File "/home/draxtik20/Federated_Learning_CDI/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filejdqiy1vn.py", line 104, in tf__call
        ag__.if_stmt(ag__.ld(self)._semantics is None, if_body_2, else_body_2, get_state_3, set_state_3